In [1]:
!pip install diffusers transformers sentencepiece accelerate

!pip install torch
!pip install transformers[torch]
!pip install diffusers PIL
!pip install accelerate
!pip install --upgrade transformers

!pip install transformers_stream_generator

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [2]:

from PIL import Image
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available


!pip install ipywidgets --upgrade
!pip install jupyter --upgrade
!pip install tqdm notebook


True


In [3]:
!pip install opencv-python


In [4]:
# Load model directly
from transformers import AutoModel
model = AutoModel.from_pretrained("openbmb/MiniCPM-V-2_6", trust_remote_code=True)

2024-12-05 18:33:56.388060: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 18:33:56.402351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733403836.419805  135887 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733403836.425026  135887 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 18:33:56.443439: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
import torch
import cv2
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json

model_name = 'openbmb/MiniCPM-V-2_6'
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    attn_implementation='sdpa', 
    torch_dtype=torch.bfloat16
).eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

image_folder_path = '/home/interiit/mp3/MG/images_for_VLM/256'  # Replace with the actual path to your image folder
output_file_path = '/home/interiit/mp3/assessing_VLMs/artifact_analysis.json'  # Output JSON file

# List all images in the folder
image_files = [f for f in os.listdir(image_folder_path) if f.endswith(('png', 'jpg', 'jpeg'))]

# Artifact detection question
question = """
List of artifacts:
- Inconsistent object boundaries
- Discontinuous surfaces
- Non-manifold geometries in rigid structures
- Floating or disconnected components
- Asymmetric features in naturally symmetric objects
- Misaligned bilateral elements in animal faces
- Irregular proportions in mechanical components
- Texture bleeding between adjacent regions
- Texture repetition patterns
- Over-smoothing of natural textures
- Artificial noise patterns in uniform surfaces
- Unrealistic specular highlights
- Inconsistent material properties
- Metallic surface artifacts
- Dental anomalies in mammals
- Anatomically incorrect paw structures
- Improper fur direction flows
- Unrealistic eye reflections
- Misshapen ears or appendages
- Impossible mechanical connections
- Inconsistent scale of mechanical parts
- Physically impossible structural elements
- Inconsistent shadow directions
- Multiple light source conflicts
- Missing ambient occlusion
- Incorrect reflection mapping
- Incorrect perspective rendering
- Scale inconsistencies within single objects
- Spatial relationship errors
- Depth perception anomalies
- Over-sharpening artifacts
- Aliasing along high-contrast edges
- Blurred boundaries in fine details
- Jagged edges in curved structures
- Random noise patterns in detailed areas
- Loss of fine detail in complex structures
- Artificial enhancement artifacts
- Incorrect wheel geometry
- Implausible aerodynamic structures
- Misaligned body panels
- Impossible mechanical joints
- Distorted window reflections
- Anatomically impossible joint configurations
- Unnatural pose artifacts
- Biological asymmetry errors
- Regular grid-like artifacts in textures
- Repeated element patterns
- Systematic color distribution anomalies
- Frequency domain signatures
- Color coherence breaks
- Unnatural color transitions
- Resolution inconsistencies within regions
- Unnatural Lighting Gradients
- Incorrect Skin Tones
- Fake depth of field
- Abruptly cut off objects
- Glow or light bleed around object boundaries
- Ghosting effects: Semi-transparent duplicates of elements
- Cinematization Effects
- Excessive sharpness in certain image regions
- Artificial smoothness
- Movie-poster like composition of ordinary scenes
- Dramatic lighting that defies natural physics
- Artificial depth of field in object presentation
- Unnaturally glossy surfaces
- Synthetic material appearance
- Multiple inconsistent shadow sources
- Exaggerated characteristic features
- Impossible foreshortening in animal bodies
- Scale inconsistencies within the same object class

From the set of artifacts given, analyze the image and give the artifacts present in the image (JSON only) which should include the name of the artifact and a brief explanation about the artifact (which includes the position of artifact in the image) as to why the image has this artifact. Provide only the JSON output
"""

# Initialize a list to store results for all images
all_results = []

# Loop over each image in the folder
for image_file in image_files:
    image_path = os.path.join(image_folder_path, image_file)
    imag = Image.open(image_path).convert('RGB')

    image = imag.resize((256, 256), Image.Resampling.LANCZOS)

    msgs = [{'role': 'user', 'content': [image, question]}]

    res = model.chat(
        image=image,
        msgs=msgs,
        tokenizer=tokenizer
    )

    # Collect the result for the current image
    image_result = {
        "image_name": image_file,
        "artifact_analysis": res
    }

    all_results.append(image_result)

    print(f"Artifact analysis for {image_file} collected.")

# Save the results for all images in a single JSON file
with open(output_file_path, 'w') as json_file:
    json.dump(all_results, json_file, indent=4)

print(f"All artifact analysis saved to {output_file_path}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Artifact analysis for image1.jpg collected.
Artifact analysis for image6.jpg collected.
Artifact analysis for image7.jpg collected.
Artifact analysis for image8.jpg collected.
Artifact analysis for image9.jpg collected.
Artifact analysis for image3.jpg collected.
Artifact analysis for image2.jpg collected.
Artifact analysis for image4.jpg collected.
Artifact analysis for image5.jpg collected.
All artifact analysis saved to /home/interiit/mp3/assessing_VLMs/artifact_analysis.json
